In [1]:
import argparse
import os
import sys
import time
import torch
import torch.nn.functional as F
import torchvision
from tqdm import tqdm
from torchsummary import summary
import tabulate
import torchvision.models as models
import cv2
import numpy as np

In [7]:
#resnet = models.resnet50(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to C:\Users\praty/.cache\torch\checkpoints\resnet50-19c8e357.pth
100.0%


In [8]:
#summary(resnet, input_size=(3,460,700))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 230, 350]           9,408
       BatchNorm2d-2         [-1, 64, 230, 350]             128
              ReLU-3         [-1, 64, 230, 350]               0
         MaxPool2d-4         [-1, 64, 115, 175]               0
            Conv2d-5         [-1, 64, 115, 175]           4,096
       BatchNorm2d-6         [-1, 64, 115, 175]             128
              ReLU-7         [-1, 64, 115, 175]               0
            Conv2d-8         [-1, 64, 115, 175]          36,864
       BatchNorm2d-9         [-1, 64, 115, 175]             128
             ReLU-10         [-1, 64, 115, 175]               0
           Conv2d-11        [-1, 256, 115, 175]          16,384
      BatchNorm2d-12        [-1, 256, 115, 175]             512
           Conv2d-13        [-1, 256, 115, 175]          16,384
      BatchNorm2d-14        [-1, 256, 1

In [9]:
# os.mkdir('./data')

In [10]:
#resnext = models.resnext50_32x4d(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to C:\Users\praty/.cache\torch\checkpoints\resnext50_32x4d-7cdf4587.pth
100.0%


In [11]:
#summary(resnext, input_size=(3,460,700))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 230, 350]           9,408
       BatchNorm2d-2         [-1, 64, 230, 350]             128
              ReLU-3         [-1, 64, 230, 350]               0
         MaxPool2d-4         [-1, 64, 115, 175]               0
            Conv2d-5        [-1, 128, 115, 175]           8,192
       BatchNorm2d-6        [-1, 128, 115, 175]             256
              ReLU-7        [-1, 128, 115, 175]               0
            Conv2d-8        [-1, 128, 115, 175]           4,608
       BatchNorm2d-9        [-1, 128, 115, 175]             256
             ReLU-10        [-1, 128, 115, 175]               0
           Conv2d-11        [-1, 256, 115, 175]          32,768
      BatchNorm2d-12        [-1, 256, 115, 175]             512
           Conv2d-13        [-1, 256, 115, 175]          16,384
      BatchNorm2d-14        [-1, 256, 1

In [12]:
# torch.save(resnet, './resnet.pt')
# torch.save(resnext, './resnext.pt')

In [2]:
resnet = torch.load('./resnet.pt')

In [3]:
resnet = torch.nn.Sequential(*(list(resnet.children())[:-1]))

In [4]:
resnet.cuda()

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [5]:
file_path = './BreaKHis_v1/histology_slides/breast/'
dest_path = './data/resnet/' # './data/40X'

In [21]:
# def transform(f):
#     files = os.listdir(f)
#     for file in files:
#         if os.path.isdir(f+file):
#             if file != '40X':
#                 transform(f+file+'/')
#             else:
#                 file = f+file + '/'
#                 images = os.listdir(file)
#                 for _img in images:
#                     img = cv2.imread(file+_img)
#                     img = np.reshape(img, (3,460,700))
#                     img = torch.tensor(np.expand_dims(img, axis=0))
#                     img = img.type(torch.FloatTensor)
#                     img = img.cuda()
#                     feature = resnet(img)
#                     print(feature.shape)

In [16]:
def transform(f, model):
    files = os.listdir(f)
    for file in files:
        if os.path.isdir(f+file):
            if file not in ['40X', '100X', '200X', '400X']:
                transform(f+file+'/', model)
            else:
                filePTH = f+file + '/'
                images = os.listdir(filePTH)
                for _img in images:
                    img = cv2.imread(filePTH+_img)
                    img = cv2.resize(img, (700,460))
                    img = np.reshape(img, (3,460,700))
                    img = torch.tensor(np.expand_dims(img, axis=0))
                    img = img.type(torch.FloatTensor)
                    img = img.cuda()
                    feature = model(img).cpu().detach().numpy()
                    
                    destPTH = dest_path+file+'/'+_img[:-4]
                    np.save(destPTH, feature[0])

In [7]:
transform(file_path, resnet)

In [17]:
resnext = torch.load('./resnext.pt')

In [18]:
resnext = torch.nn.Sequential(*(list(resnext.children())[:-1]))

In [19]:
resnext.cuda()

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1,

In [20]:
file_path = './BreaKHis_v1/histology_slides/breast/'
dest_path = './data/resnext/'

In [21]:
transform(file_path, resnext)